<a href="https://colab.research.google.com/github/onewon1234/AI_DL_Project/blob/main/BERT_Code_Final_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm

import torch
from torch.utils.data import Dataset
import torch.nn as nn

In [ ]:
import transformers
import torch

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback

import itertools
from itertools import permutations, combinations
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
import os
os.environ["WANDB_DISABLED"] = "true"
import matplotlib.pyplot as plt


In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = '/content/drive/MyDrive/data/daycon_sentence/train.csv'
test_path = '/content/drive/MyDrive/data/daycon_sentence/test.csv'

In [ ]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
train.head()

,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2
1,TRAIN_0001,줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를...,"결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.",생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.,"이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을...",0,3,2,1
2,TRAIN_0002,"마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는...",재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.,물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.,키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.,3,2,1,0
3,TRAIN_0003,책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.,"정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.",종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.,"필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할...",2,0,1,3
4,TRAIN_0004,"인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성...",인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도...,"결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...",이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기...,1,3,0,2


## Pairwise 쌍 만들기

In [ ]:
def create_pairwise_data(df):
    pairs, labels = [], []
    for _, row in df.iterrows():
        sentences = [row[f'sentence_{i}'] for i in range(4)]
        answers = [row[f'answer_{i}'] for i in range(4)]
        for i, j in combinations(range(4), 2):
            pairs.append((sentences[i], sentences[j]))
            labels.append(1 if answers[i] < answers[j] else 0)
    return pairs, labels


## Dataset 클래스 정의

In [ ]:
class SentencePairDataset(Dataset):
    def __init__(self, pairs, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(
            [p[0] for p in pairs], [p[1] for p in pairs],
            truncation=True, padding='max_length', max_length=max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'token_type_ids': self.encodings.get('token_type_ids', torch.zeros_like(self.encodings['input_ids']))[idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

## 문장 순서 예측 함수 (pairwise 결과 기반 순서 재구성)

In [ ]:
def predict_sentence_order(model, tokenizer, sentences):
    """
    4개의 문장이 주어졌을 때, 가능한 문장쌍 (i, j)에 대해 모델이 i가 j보다 앞선다고 예측한 비율을 바탕으로 전체 순서를 추론합니다.
    """
    model.eval()
    device = next(model.parameters()).device
    scores = np.zeros((4, 4))

    for i, j in permutations(range(4), 2):
        inputs = tokenizer(
            sentences[i], sentences[j],
            truncation=True, padding='max_length', max_length=128,
            return_tensors='pt'
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            logits = model(**inputs).logits
            pred = torch.argmax(logits, dim=1).item()
            if pred == 1:
                scores[i][j] += 1

    order = sorted(range(4), key=lambda x: -sum(scores[x]))
    return order

## 평가 메트릭 함수 정의

In [ ]:
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds)
    }

## 학습 및 평가 실행

In [ ]:
model_name = 'klue/bert-base'  # 또는 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

## 데이터 로드 및 전처리

In [ ]:
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

train_pairs, train_labels = create_pairwise_data(train_df)
val_pairs, val_labels = create_pairwise_data(val_df)

train_dataset = SentencePairDataset(train_pairs, train_labels, tokenizer)
val_dataset = SentencePairDataset(val_pairs, val_labels, tokenizer)

In [ ]:
# 모델 및 학습 설정
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    save_total_limit=1,
    logging_dir='./logs',
    logging_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


## 학습 및 평가

In [ ]:
train_result = trainer.train()
eval_result = trainer.evaluate()
print("\n[Validation Metrics]")
for k, v in eval_result.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.642200,0.638647,0.648312,0.588220,0.720976,0.496751
2,0.579100,0.650414,0.656809,0.639190,0.682350,0.601165


## 시각화: 로그 히스토리 기반 loss 시각화

In [ ]:
train_log = trainer.state.log_history
train_losses = [x['loss'] for x in train_log if 'loss' in x]
eval_losses = [x['eval_loss'] for x in train_log if 'eval_loss' in x]

plt.figure(figsize=(8, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(eval_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

NameError: name 'trainer' is not defined

In [ ]:
# 예측 예시
example = [
    "문장 A입니다.",
    "문장 B입니다.",
    "문장 C입니다.",
    "문장 D입니다."
]
predicted_order = predict_sentence_order(model, tokenizer, example)
print("예측된 순서:", predicted_order)


In [ ]:
import nbformat

# 정확한 경로 입력
path = "/content/drive/MyDrive/AI를 위한 딥러닝/BERT_Code_Final.ipynb"
output_path = "/content/drive/MyDrive/AI를 위한 딥러닝/BERT_Code_Final_clean.ipynb"

# 노트북 로드
with open(path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# 1. 노트북 metadata에서 widgets 제거
nb['metadata'].pop('widgets', None)

# 2. 셀 내부 metadata에서도 widgets 제거
for cell in nb['cells']:
    if 'metadata' in cell and 'widgets' in cell['metadata']:
        del cell['metadata']['widgets']

# 저장
with open(output_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("✅ widgets 관련 메타데이터 모두 제거 완료")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AI를 위한 딥러닝/BERT_Code_Final.ipynb'